<div align=center>

# Example Jupyter Notebook

</div>

## Check if the kernel is up and working

In [ ]:
print("Kernel works!")

### Import required packages

In [29]:
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import torch

## Perform basic operations to test installed packages

In [ ]:
# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Check if GPU is available
gpu_available = torch.cuda.is_available()

if gpu_available:
    print("GPUs are available!")
    print("Amount of available GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} name:", torch.cuda.get_device_name(i))
else:
    print("GPU is not available. Will use CPU instead")

## Setup the workflow
<div align=center>

![pytorch_workflow](pytorch_workflow.png)

</div>

### 1. Get data ready

#### Define a plotting function

In [31]:
def plot_data(train_data, test_data, train_labels, test_labels, title, predictions=None):
    """
    Plots a given function.

    Parameters:
        func: A function to plot (takes a numpy array as input).
        title: Title of the plot.
        x_range: Tuple indicating the range of x values (default is (-10, 10)).
        num_points: Number of points to plot (default is 100).
    """

    # Set the font to be JetBrains Mono
    font_path = Path("../fonts/JetBrainsMono-Regular.ttf")

    # Set the dark mode
    plt.style.use('dark_background')

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(train_data, train_labels, c="green", label="Training data")
    plt.scatter(train_data, train_labels, c="green")

    plt.plot(test_data, test_labels, c="orange", label="Testing data")
    plt.scatter(test_data, test_labels, c="orange")

    if predictions is not None:
        plt.plot(test_data, predictions, c="red", label="Model predictions")
        plt.scatter(test_data, predictions, c="red")

    # Setting up axes
    plt.title(title, font=font_path, fontsize=16)
    plt.xlabel('X', font=font_path, fontsize=14)
    plt.ylabel('Y', font=font_path, fontsize=14)

    # Add grid and axes lines
    plt.axhline(0, color='white', lw=0.5, ls='--')
    plt.axvline(0, color='white', lw=0.5, ls='--')
    plt.grid(color='gray', linestyle='--', linewidth=0.5)

    # Customize ticks
    plt.xticks(font=font_path)
    plt.yticks(font=font_path)

    # Add a legend
    plt.legend(prop=font_path)

#### Generate example data

In [ ]:
# Set the random seed for reproducibility
torch.manual_seed(42)

# Number of samples
num_samples = 42

# Generate random input features (x) and corresponding labels (y)
X = torch.linspace(0, 1, num_samples).unsqueeze(1)  # Shape (25, 1)
# Let's assume a linear relationship: y = 0.7 * x + 0.3
y = 0.7 * X + 0.3

data_df = pd.DataFrame({'X': X.numpy().flatten(), 'y': y.numpy().flatten()})

# Create train/test split
train_split = int(0.5 * len(X))  # 50% of data used for training set, 50% for testing
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

# Convert to pandas DataFrame
train_df = pd.DataFrame({'X': X_train.numpy().flatten(), 'y': y_train.numpy().flatten()})
test_df = pd.DataFrame({'X': X_test.numpy().flatten(), 'y': y_test.numpy().flatten()})

data_df

### 2. Build a model

#### Build a model class

In [33]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = torch.nn.Parameter(torch.randn(1))
        self.bias = torch.nn.Parameter(torch.randn(1))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

#### Build a training loop

In [ ]:
class Color:
    RED = '\033[91m'
    ORANGE = '\033[93m'
    GREEN = '\033[92m'
    RESET = '\033[0m'

# Initialize model, loss function, optimizer, and scheduler
model = LinearRegressionModel()
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

epochs = 2**10
interval = max(1, epochs // 10000)
thresholds = [0.01, 0.1]  # 95% and 50% thresholds

for epoch in range(epochs):
    model.train()
    loss = criterion(model(X_test), y_test)

    # Color assignment based on loss
    if loss.item() < thresholds[0]:
        loss_color = Color.GREEN
    elif loss.item() < thresholds[1]:
        loss_color = Color.ORANGE
    else:
        loss_color = Color.RED

    if epoch % interval == 0:
        print(f"epoch: {epoch} ({(epoch + 1) / epochs * 100:.2f}% complete) | loss: {loss_color}{loss.item():.12f}{Color.RESET}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step(loss.item())

#### View model prediction performance

In [ ]:
with torch.inference_mode():
    y_preds = model(X_test)
plot_data(X_train, X_test, y_train, y_test, "Model performance", y_preds)